# imports

In [33]:
import os
from glob import glob
import pandas as pd
from load_configs import openai_api_key
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.tools import tool


from typing import (
    List, Sequence, Tuple, Callable, Optional, Union, Literal,
    Dict, cast, TYPE_CHECKING, Any
)


# csv data

In [20]:
data_path = os.path.join("../data")
os.path.exists(data_path)
csv_files = glob(f"{data_path}/*.csv")
print(csv_files)
sfile = csv_files[0]
df_small = pd.read_csv(sfile)
print('small csv:\n', df_small.count())
lfile = csv_files[1]
df_large = pd.read_csv(lfile)
print()
print('large csv:\n', df_large.count())

['../data/country_eng.csv', '../data/custom_2020.csv', '../data/Mobile-Price-Prediction-cleaned_data.csv']
small csv:
 Unnamed: 0      232
Country         232
Country_name    232
Area            232
dtype: int64

large csv:
 exp_imp    3299804
Year       3299804
month      3299804
ym         3299804
Country    3299804
Custom     3299804
hs2        3299804
hs4        3299804
hs6        3299804
hs9        3299804
Q1         3299804
Q2         3299804
Value      3299804
dtype: int64


# chat model

In [22]:
chatmodel = ChatOpenAI(api_key=openai_api_key.get_secret_value(), 
                   model="gpt-4o-mini",
                   # model="gpt-4o-mini-2024-07-18",                   
                   # model="gpt-4o"
                  )
print(chatmodel)
chatmodel.invoke('hi')

client=<openai.resources.chat.completions.Completions object at 0x7fc5053c35e0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc5053c5f70> model_name='gpt-4o-mini' openai_api_key=SecretStr('**********') openai_proxy=''


AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-00780032-9b0b-46d7-8482-549de8a284a4-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

# How csv agent worked

In [23]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
agent = create_csv_agent(
    chatmodel,
    sfile,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    verbose=True
)
agent.run("how many rows are there?")

/mnt/d/myDev/llmapps/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


232There are 232 rows in the dataframe.

> Finished chain.


'There are 232 rows in the dataframe.'

csv_agent - https://github.com/langchain-ai/langchain/blob/master/libs/experimental/langchain_experimental/agents/agent_toolkits/csv/base.py

In [30]:
from io import IOBase
from langchain.agents.agent import AgentExecutor
from langchain_core.language_models import LanguageModelLike
from langchain_experimental.agents.agent_toolkits.pandas.base import (
    create_pandas_dataframe_agent,
)

def create_csv_agent(
    llm: LanguageModelLike,
    path: Union[str, IOBase, List[Union[str, IOBase]]],
    pandas_kwargs: Optional[dict] = None,
    **kwargs: Any,
) -> AgentExecutor:
    try:
        import pandas as pd
    except ImportError:
        raise ImportError(
            "pandas package not found, please install with `pip install pandas`."
        )

    _kwargs = pandas_kwargs or {}
    if isinstance(path, (str, IOBase)):
        df = pd.read_csv(path, **_kwargs)
    elif isinstance(path, list):
        df = []
        for item in path:
            if not isinstance(item, (str, IOBase)):
                raise ValueError(f"Expected str or file-like object, got {type(path)}")
            df.append(pd.read_csv(item, **_kwargs))
    else:
        raise ValueError(f"Expected str, list, or file-like object, got {type(path)}")
    return create_pandas_dataframe_agent(llm, df, **kwargs)


In [ ]:
- after inspecting the pandas agent we see that padas agent is calling the agent executor directly 
- we want to beak it down to create the agent first 
- then manually passing it to agentexecutor 

In [35]:
from langchain.agents import (
    AgentType,
    create_openai_tools_agent,
    create_react_agent,
    create_tool_calling_agent,
)
from langchain.agents.agent import (
    AgentExecutor,
    BaseMultiActionAgent,
    BaseSingleActionAgent,
    RunnableAgent,
    RunnableMultiActionAgent,
)
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS
from langchain.agents.openai_functions_agent.base import (
    OpenAIFunctionsAgent,
    create_openai_functions_agent,
)
from langchain_core.callbacks import BaseCallbackManager
from langchain_core.language_models import BaseLanguageModel, LanguageModelLike
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    BasePromptTemplate,
    ChatPromptTemplate,
    PromptTemplate,
)
from langchain_core.tools import BaseTool
from langchain_core.utils.interactive_env import is_interactive_env

from langchain_experimental.agents.agent_toolkits.pandas.prompt import (
    FUNCTIONS_WITH_DF,
    FUNCTIONS_WITH_MULTI_DF,
    MULTI_DF_PREFIX,
    MULTI_DF_PREFIX_FUNCTIONS,
    PREFIX,
    PREFIX_FUNCTIONS,
    SUFFIX_NO_DF,
    SUFFIX_WITH_DF,
    SUFFIX_WITH_MULTI_DF,
)
from langchain_experimental.tools.python.tool import PythonAstREPLTool



In [79]:
mydata = {}
mydata["mylist"] = [i for i in range(10)]
ptool = PythonAstREPLTool(locals=mydata)
print(ptool.args)
print(ptool.dict())
print(ptool({'query': 'len(mylist)'}))
mydata["mylist"] = [i for i in range(10)]
# ptool.response_format='content_and_artifact' 
res = ptool({'query': 'mylist, mylist[::-1]'})  ### mylist[start:stop:step]
print(res)
mydata["mylist"] = [i for i in range(10)]
ptool.response_format='content_and_artifact' 
res = ptool({'query': 'mylist, mylist[::-1]'})  ### mylist[start:stop:step]
print(res)
# PythonAstREPLTool.
mydata["mydf"] = df_large
ptool = PythonAstREPLTool(locals=mydata)
res = ptool({'query': 'mydf[:2]'})  ### mylist[start:stop:step]
print(res)

{'query': {'title': 'Query', 'description': 'code snippet to run', 'type': 'string'}}
{'name': 'python_repl_ast', 'description': 'A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.', 'args_schema': <class 'langchain_experimental.tools.python.tool.PythonInputs'>, 'return_direct': False, 'verbose': False, 'tags': None, 'metadata': None, 'handle_tool_error': False, 'handle_validation_error': False, 'response_format': 'content', 'globals': {}, 'locals': {'mylist': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}, 'sanitize_input': True}
10
([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [9, 8, 7, 6, 5, 4, 3, 2, 1, 0])
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
   exp_imp  Year  month      ym  Country  Custom  hs2  hs4  hs6  hs9  Q1  \
0        1  2020      1  202001      103     100    0    0    0  190   0   
1        1  2020      1  202001      105     100    0    0

In [81]:
tool_call_data = {
        "name": "calculate_revenue_tool",
        "args": {'query': 'mydf[:2]'},
        "id": "1" , ## required field
        "type": "tool_call", ## required field
    }
res = ptool.invoke(tool_call_data)  ### mylist[start:stop:step]
print(res)

content='   exp_imp  Year  month      ym  Country  Custom  hs2  hs4  hs6  hs9  Q1  \\\n0        1  2020      1  202001      103     100    0    0    0  190   0   \n1        1  2020      1  202001      105     100    0    0    0  190   0   \n\n        Q2    Value  \n0   526875  1150084  \n1  1241751  3830662  ' name='python_repl_ast' tool_call_id='1'


In [90]:
df_large.head()

,exp_imp,Year,month,ym,Country,Custom,hs2,hs4,hs6,hs9,Q1,Q2,Value
0,1,2020,1,202001,103,100,0,0,0,190,0,526875,1150084
1,1,2020,1,202001,105,100,0,0,0,190,0,1241751,3830662
2,1,2020,1,202001,106,100,0,0,0,190,0,505391,1530045
3,1,2020,1,202001,107,100,0,0,0,190,0,1852,5150
4,1,2020,1,202001,108,100,0,0,0,190,0,455623,5664430


In [92]:
df_large.groupby('Country')['Value'].sum().nlargest(10).reset_index()

,Country,Value
0,105,29434277157
1,304,18241015411
2,106,6914098260
3,103,6854247392
4,111,4754737641
5,601,4637475639
6,110,3801548598
7,213,3762144099
8,108,3158800299
9,113,2754663778


The above result shows that the PythonAstREPLTool  never produces the artifact


- When the result  from the tool is very large how to avoid the model input length error
- Given the df dimensions, example data generatea df_query to get the answer of the users question
- while geneating the df_query analyze if it is going to produce a single numerical value or a single string as output_result
- if the output of the df_query seems to produce an output_df or a pandas series which is slice of the df, generate an intermediate query to assess the length of the output_df or output_series
- if the output_df or or output_series row or column length is more than 50, modify the df_query to df_query_to_summarize, to produce a summary or a numerical value
- otherwise give the df_query  without modification
- when the tool result is an error or a stack trace of the query , do not show theose traces. Simply say I could not answer this due to an error. Try other question. 

Example 1:
- Query: give me the list of all the cities 
- Thought: This query seems to produce a large series. Let me first use the tool1 to get the count of city
- Action: tool1: {'query': 'df.Country.size'}
- Observation: There are 3200000 rows in this output which is more than 50
- Thought: I should call tool1 with the first 50 country 
- Action: tool1: {'query': 'df.Country[:50]'}
- Observation: I got the total count as well as the first 50 name of the country
- Answer: There are total 3.2 millon rows for the country in this database. Here are the name of the first 50 country

Example 2:

- Query: show me the top 10 countery by value
- Thought: this query will produce a small subset of the rows, I will use the tool directly with the query without asseing the size of the output
- Action: tool1: {'query': 'df_large.groupby('Country')['Value'].sum().nlargest(10).reset_index()'}
- Observation: I got the result 
- Answer: The top countries by values are as follows , country , value


Now we should go ahead and check where to use this prompt 

In [93]:
def create_pandas_dataframe_agent(
    llm: LanguageModelLike,
    df: Any,
    agent_type: Union[
        AgentType, Literal["openai-tools", "tool-calling"]
    ] = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    
    prefix: Optional[str] = None,
    suffix: Optional[str] = None,
    input_variables: Optional[List[str]] = None,
    verbose: bool = False,
    return_intermediate_steps: bool = False,
    max_iterations: Optional[int] = 15,
    max_execution_time: Optional[float] = None,
    early_stopping_method: str = "force",
    agent_executor_kwargs: Optional[Dict[str, Any]] = None,
    include_df_in_prompt: Optional[bool] = True,
    number_of_head_rows: int = 5,
    extra_tools: Sequence[BaseTool] = (),
    engine: Literal["pandas", "modin"] = "pandas",
    allow_dangerous_code: bool = False,
    **kwargs: Any,
) -> AgentExecutor:
    df_locals = {}
    if isinstance(df, list):
        for i, dataframe in enumerate(df):
            df_locals[f"df{i + 1}"] = dataframe
    else:
        df_locals["df"] = df
    tools = [PythonAstREPLTool(locals=df_locals)] + list(extra_tools)
    
create_pandas_dataframe_agent(chatmodel, df_small)

# PythonAstREPLTool HLEP

In [53]:
help(PythonAstREPLTool)

Help on class PythonAstREPLTool in module langchain_experimental.tools.python.tool:

class PythonAstREPLTool(langchain_core.tools.BaseTool)
 |  PythonAstREPLTool(*, name: str = 'python_repl_ast', description: str = 'A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.', args_schema: Type[pydantic.v1.main.BaseModel] = <class 'langchain_experimental.tools.python.tool.PythonInputs'>, return_direct: bool = False, verbose: bool = False, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, handle_tool_error: Union[bool, str, Callable[[langchain_core.tools.ToolException], str], No